In [1]:
import rioxarray as rxr
import os
from rasterio.plot import show
from PIL import Image
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd
import rasterio.features
from shapely.geometry import shape
import pandas as pd

In [7]:
folder = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb"

list_bands = ['02', '03', '04', '05', '06', '07', '08', '8A', '11', '12']

In [9]:
def normalize_and_save_band(band, output_folder):
    with rasterio.open(band) as src:
        normalized_acquisitions = []
        for acquisition in range(1, src.count + 1):
            print(f"Acquisition {acquisition}/{src.count}")
            data_array = src.read(acquisition)
            min_val = (np.percentile(data_array, 2))
            max_val = (np.percentile(data_array, 98))
            norm_band = np.clip((data_array - min_val) / (max_val - min_val), 0, 1).astype('float32')
            print(f"Min: {min_val}, Max: {max_val}, Mean: {np.mean(norm_band)}")

            # Append the normalized acquisition to the list
            normalized_acquisitions.append(norm_band)

        # Stack the acquisitions along a new dimension (time)
        normalized_band = np.stack(normalized_acquisitions, axis=0)

        # Define the output file path for the normalized band
        output_path = os.path.join(output_folder, f'normalized_{os.path.basename(band)}.tif')

        # Create a new GeoTIFF file with a custom profile and write the normalized data
        profile = src.profile.copy()
        profile.update(count=src.count, dtype='float32')  # Set the count to the number of acquisitions
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(normalized_band)

In [10]:
output_band_folder = os.path.join(folder, 'normalized_gathered_output')
os.makedirs(output_band_folder, exist_ok=True)

In [15]:
for band in list_bands:
    print(f"Band {band}")
    band_file = os.path.join(folder, 's2_2020_B' + band + '.tif')
    normalize_and_save_band(band_file, output_band_folder)

Band 02
Acquisition 1/73
Min: 551.0, Max: 1180.0, Mean: 0.42213112115859985
Acquisition 2/73
Min: 601.0, Max: 1206.0, Mean: 0.42730656266212463
Acquisition 3/73
Min: 491.0, Max: 1218.0, Mean: 0.49325627088546753
Acquisition 4/73
Min: 781.0, Max: 1368.0, Mean: 0.41971680521965027
Acquisition 5/73
Min: 657.0, Max: 1362.0, Mean: 0.4103529751300812
Acquisition 6/73
Min: 463.0, Max: 1318.0, Mean: 0.404828816652298
Acquisition 7/73
Min: 457.0, Max: 1362.0, Mean: 0.39928632974624634
Acquisition 8/73
Min: 668.0, Max: 1302.0, Mean: 0.3932023346424103
Acquisition 9/73
Min: 728.0, Max: 1472.0, Mean: 0.4012269973754883
Acquisition 10/73
Min: 745.0, Max: 1424.0, Mean: 0.39201831817626953
Acquisition 11/73
Min: 588.0, Max: 1388.0, Mean: 0.3870910108089447
Acquisition 12/73
Min: 807.0, Max: 1532.0, Mean: 0.3904193043708801
Acquisition 13/73
Min: 1013.0, Max: 1562.0, Mean: 0.40821096301078796
Acquisition 14/73
Min: 910.0, Max: 1625.0, Mean: 0.4155017137527466
Acquisition 15/73
Min: 828.0, Max: 1716.0,